# Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial.distance import euclidean
from scipy.stats import mode

Loading dataset

In [ ]:
dataset = load_wine()

Making dataframe from loaded dataset

In [ ]:
df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
df['class'] = dataset.target
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


Printing basic information about dataset

In [ ]:
df.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,1.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,2.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,2.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  class

Splitting into labels and features

In [ ]:
label = 'class'
X = df.drop([label], axis=1)
y = df[label]
print(X.head())
print(y.head())

   alcohol  malic_acid   ash  ...   hue  od280/od315_of_diluted_wines  proline
0    14.23        1.71  2.43  ...  1.04                          3.92   1065.0
1    13.20        1.78  2.14  ...  1.05                          3.40   1050.0
2    13.16        2.36  2.67  ...  1.03                          3.17   1185.0
3    14.37        1.95  2.50  ...  0.86                          3.45   1480.0
4    13.24        2.59  2.87  ...  1.04                          2.93    735.0

[5 rows x 13 columns]
0    0
1    0
2    0
3    0
4    0
Name: class, dtype: int64


Splitting into train and test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Creating functions that performs K-Nearest-Neighbors algorithm


In [ ]:
def KNN(dataPoint, X, y, n_neighbors=5):
  distances = np.array([])
  for index, point in enumerate(X.values):
    distances = np.append(distances, 
                          np.array([index, euclidean(dataPoint, point)]))
    distances = distances.reshape(-1, 2)
    distances = distances[distances[:,1].argsort()]
  neighbors = distances[:n_neighbors, 0].astype('int32')
  classes = y.values[neighbors]
  dataPointClass = mode(classes)[0][0]
  return dataPointClass


Testing the function for k = 5

In [ ]:
preds = np.array([], dtype='int8')
for point in X_test.values:
  preds = np.append(preds, KNN(point, X_train, y_train, n_neighbors=5))

print(f'Predicted classes: {preds}')
print(f'Actual classes: {y_test.values}')

print(f'''\nPredicted correctly = {len(np.flatnonzero(preds == y_test.values))},
Number of datapoints = {len(y_test.values)}
Acc = {len(np.flatnonzero(preds == y_test.values)) / len(y_test.values)}''')

Predicted classes: [2 0 2 1 0 1 1 2 1 2 0 1 0 1 0 2 1 2 0 0 0 2 0 2 2 1 2 2 1 2 2 2 0 1 1 0]
Actual classes: [2 0 1 1 0 1 1 1 2 1 1 1 2 1 0 1 1 1 0 0 0 2 0 0 2 2 1 2 1 1 2 1 0 1 2 0]

Predicted correctly = 22,
Number of datapoints = 36
Acc = 0.6111111111111112


Using sklearn's KNN classifier to check the results

In [ ]:
for k in [3, 5, 7, 9, 11, 13]:
  model = KNeighborsClassifier(n_neighbors=k)
  model.fit(X_train, y_train)
  print(f'Neighbors = {k}, acc = {model.score(X_test, y_test)}')


Neighbors = 3, acc = 0.6388888888888888
Neighbors = 5, acc = 0.6111111111111112
Neighbors = 7, acc = 0.6111111111111112
Neighbors = 9, acc = 0.5833333333333334
Neighbors = 11, acc = 0.5833333333333334
Neighbors = 13, acc = 0.6388888888888888


Testing my function with k = 9

In [ ]:
preds = np.array([], dtype='int8')
for point in X_test.values:
  preds = np.append(preds, KNN(point, X_train, y_train, n_neighbors=9))

print(f'Predicted classes: {preds}')
print(f'Actual classes: {y_test.values}')

print(f'''\nPredicted correctly = {len(np.flatnonzero(preds == y_test.values))},
Number of datapoints = {len(y_test.values)}
Acc = {len(np.flatnonzero(preds == y_test.values)) / len(y_test.values)}''')

Predicted classes: [1 0 2 1 0 1 1 1 1 2 0 2 0 1 0 2 1 2 0 0 0 2 0 2 2 1 1 1 1 1 1 2 0 1 1 0]
Actual classes: [2 0 1 1 0 1 1 1 2 1 1 1 2 1 0 1 1 1 0 0 0 2 0 0 2 2 1 2 1 1 2 1 0 1 2 0]

Predicted correctly = 21,
Number of datapoints = 36
Acc = 0.5833333333333334


That is just a basic algorith implementation. As we can see it is not good for our dataset. On the Internet we can find that for example Random Forest algorithm performs much better. I'll check it below

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
print(f'Acc = {model.score(X_test, y_test)}')

Acc = 0.9722222222222222


Accuracy seems to be very high, so it is better to use this algorithm for wine dataset.